In [3]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('dataset.csv')

# Model 1: Baseline Linear Model
def baseline_linear_model(df, alpha=0.1, base_price=10):
    prices = [base_price]
    capacity = df['Capacity'].iloc[0]

    for i in range(1, len(df)):
        occupancy_ratio = df['Occupancy'].iloc[i] / capacity
        new_price = prices[-1] + alpha * occupancy_ratio
        prices.append(new_price)

    return prices

# Apply model
df['BaselinePrice'] = baseline_linear_model(df)

In [4]:
# Model 2: Demand-Based Price Function
def demand_based_model(df, base_price=10, max_multiplier=2, min_multiplier=0.5):
    # Define weights for each factor
    weights = {
        'occupancy': 0.5,
        'queue': 0.3,
        'traffic': 0.2,
        'special': 0.4,
        'vehicle': 0.1
    }

    # Vehicle type weights
    vehicle_weights = {
        'car': 1.0,
        'bike': 0.8,
        'truck': 1.2,
        'cycle': 0.7
    }

    # Traffic level weights
    traffic_weights = {
        'low': -0.5,
        'average': 0,
        'high': 0.5
    }

    capacity = df['Capacity'].iloc[0]
    prices = []

    for i in range(len(df)):
        # Calculate demand components
        occupancy = (df['Occupancy'].iloc[i] / capacity) * weights['occupancy']
        queue = (df['QueueLength'].iloc[i] / 10) * weights['queue']  # assuming max queue ~10
        traffic = traffic_weights.get(df['TrafficConditionNearby'].iloc[i].lower(), 0) * weights['traffic']
        special = df['IsSpecialDay'].iloc[i] * weights['special']
        vehicle = vehicle_weights.get(df['VehicleType'].iloc[i].lower(), 1.0) * weights['vehicle']

        # Combine into demand score
        demand = occupancy + queue + traffic + special + vehicle

        # Normalize demand to [-1, 1] range
        normalized_demand = np.tanh(demand)

        # Calculate price with bounds
        price = base_price * (1 + normalized_demand)
        price = max(min(price, base_price * max_multiplier), base_price * min_multiplier)

        prices.append(price)

    return prices

# Apply model
df['DemandBasedPrice'] = demand_based_model(df)

In [5]:
# Model 3: Competitive Pricing Model (simplified)
def competitive_model(df, base_price=10, competitor_prices=None):
    if competitor_prices is None:
        # Simulate competitor prices if not provided
        competitor_prices = {
            'nearby1': {'lat': 26.145, 'long': 91.735, 'price': 9},
            'nearby2': {'lat': 26.144, 'long': 91.737, 'price': 12}
        }

    capacity = df['Capacity'].iloc[0]
    prices = []

    for i in range(len(df)):
        # Get demand-based price first
        demand_price = demand_based_model(df.iloc[i:i+1])[0]

        # Calculate weighted competitor price based on distance
        total_weight = 0
        weighted_price = 0
        our_lat = df['Latitude'].iloc[0]
        our_long = df['Longitude'].iloc[0]

        for comp in competitor_prices.values():
            # Simple distance calculation (approximate)
            dist = np.sqrt((our_lat - comp['lat'])**2 + (our_long - comp['long'])**2)
            weight = 1 / (dist + 0.001)  # avoid division by zero
            weighted_price += comp['price'] * weight
            total_weight += weight

        avg_comp_price = weighted_price / total_weight if total_weight > 0 else base_price

        # Adjust price based on competition and occupancy
        occupancy_ratio = df['Occupancy'].iloc[i] / capacity
        if occupancy_ratio > 0.9 and demand_price > avg_comp_price:
            # If nearly full and more expensive than competitors, reduce price
            final_price = min(demand_price, avg_comp_price * 0.95)
        elif occupancy_ratio < 0.5 and demand_price < avg_comp_price:
            # If lots of space and cheaper than competitors, increase price
            final_price = min(demand_price * 1.1, avg_comp_price * 0.95)
        else:
            final_price = demand_price

        prices.append(final_price)

    return prices

# Apply model
df['CompetitivePrice'] = competitive_model(df)

In [6]:
!pip install pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 

In [14]:
import pathway as pw
import pandas as pd
import numpy as np

# Fix the real-time simulation with proper schema handling
class ParkingSimulator:
    def __init__(self, df):
        self.df = df
        self.current_index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_index >= len(self.df):
            raise StopIteration
        record = self.df.iloc[self.current_index].to_dict()
        self.current_index += 1
        return record

# Correct way to define schema in Pathway
class InputSchema(pw.Schema):
    ID: int
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str

simulator = ParkingSimulator(df)

# Create Pathway table with explicit schema
table = pw.demo.replay_csv(
    simulator,
    schema=InputSchema,
    # commit every second
)

# Define pricing logic as Pathway operations
def calculate_prices(record):
    # Model 1: Baseline Linear
    base_price = 10
    capacity = record["Capacity"]
    alpha = 0.1
    price1 = base_price + alpha * (record["Occupancy"] / capacity)

    # Model 2: Demand-Based (simplified)
    demand = 0.5*(record["Occupancy"]/capacity) + 0.3*(record["QueueLength"]/10)
    price2 = base_price * (1 + np.tanh(demand))

    # Model 3: Competitive (using Model 2 as base here)
    price3 = price2  # would add competition logic in full implementation

    return price1, price2, price3

# Apply pricing models
def calculate_prices_wrapper(r):
    price1, price2, price3 = calculate_prices(r)
    return {"price1": price1, "price2": price2, "price3": price3}



# Run the pipeline
pw.run()

Output()

In [15]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot

output_notebook()

# Prepare data
source = ColumnDataSource(df)

# Create plots
p1 = figure(width=800, height=300, title="Baseline Linear Pricing")
p1.line('ID', 'BaselinePrice', source=source, line_width=2)
p1.yaxis.axis_label = "Price ($)"
p1.xaxis.axis_label = "Time (record ID)"

p2 = figure(width=800, height=300, title="Demand-Based Pricing")
p2.line('ID', 'DemandBasedPrice', source=source, line_width=2, color="green")
p2.yaxis.axis_label = "Price ($)"
p2.xaxis.axis_label = "Time (record ID)"

p3 = figure(width=800, height=300, title="Competitive Pricing")
p3.line('ID', 'CompetitivePrice', source=source, line_width=2, color="red")
p3.yaxis.axis_label = "Price ($)"
p3.xaxis.axis_label = "Time (record ID)"

# Add hover tools
for p in [p1, p2, p3]:
    p.add_tools(HoverTool(
        tooltips=[
            ("Time", "@LastUpdatedDate @LastUpdatedTime"),
            ("Occupancy", "@Occupancy/@Capacity"),
            ("Queue", "@QueueLength"),
            ("Vehicle", "@VehicleType"),
            ("Price", "$@y{0.00}")
        ]
    ))

# Show plots
show(gridplot([p1, p2, p3], ncols=1))